In [ ]:
import datetime
import apache_beam as beam
import os

In [13]:
RUNNER = "DirectRunner" #"Dataflow" #"DirectPipelineRunner"
PROJECT = 'ksalama-gcp-playground'
BUCKET = 'ksalama-gcs-demo'
REGION = 'europe-west1'

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-08-29 11:25:08.218080. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [24]:
def exract_fields(line):
    import csv
    csv_fields = next(csv.reader([line]))
    result = "{},{},{}".format(csv_fields[0], csv_fields[21], csv_fields[26])
    return result

def run_pipeline():
    
    job_name = 'extract-airports-data-{}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))

    OUTPUT_DIR = 'gs://{}/data/flights/airports'.format(BUCKET)

    options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': job_name,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'no_save_main_session': True
    }


    opts = beam.pipeline.PipelineOptions(flags=[], **options)
    p = beam.Pipeline(RUNNER,opts)

    source = 'git-repo/04_streaming/simulate/airports.csv.gz'
    
    sink = 'gs://{}/data/flights/airports/airport-locations.csv'.format(BUCKET)

    (
      p | 'Read data from text file' >> beam.io.ReadFromText(source)
        | 'Extract Fields' >> beam.Map(exract_fields)
        | 'Write csv file to GCS' >> beam.io.WriteToText(sink)
    )

    p.run()
    
    print("Pipeline done!")

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-08-29 11:36:05.010887. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [25]:
run_pipeline()

Pipeline done!


/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-08-29 11:36:05.469287. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
